# 说明

“卷积计算”要比“全连接”的复杂大的多！因此就需要非常长的时间。

有一个问题：我们只使用VGG16的卷积基的“层结构”和“权重参数”，并且我们冻结了卷积部分的权重参数让它不随进一步的训练而变化。—— 

那么：既然卷积基的参数都不变，不参与训练只负责提前特征，那它应该只用运行一次就够了！把特征提取出来并传入Dense全连接层后，往后就可以完全去掉这个卷积部分，剩下只用训练Dense全连接层的权重参数即可。—— 

因此：每张图片，只需进入一次卷积部分即可，这样就可大大避免“**重复的卷积特征提取操作**”，从而进一步提高速度！

In [2]:
import keras
from keras import layers
import numpy as np
import os
import shutil    # os和shutil用来处理文件

Using TensorFlow backend.


In [3]:
# 原始数据太多了，现在专门创建一个文件夹来存储一部分要用的训练集 + 测试集
base_dir = 'E:/Python_code/keras_total/日月光华-keras课程资料/dc/try'
train_dir = os.path.join(base_dir, 'train')
train_dir_dog = os.path.join(train_dir , 'dog')
train_dir_cat = os.path.join(train_dir , 'cat')

test_dir = os.path.join(base_dir , 'test')
test_dir_dog = os.path.join(test_dir , 'dog')
test_dir_cat = os.path.join(test_dir , 'cat')

dc_dir = 'E:/Python_code/keras_total/日月光华-keras课程资料/dc/train' # 原始数据所在路径

In [4]:
# 训练集猫狗各1000张，测试集猫狗各500张。
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
    os.mkdir(train_dir)
    os.mkdir(train_dir_dog)
    os.mkdir(train_dir_cat)
    os.mkdir(test_dir)
    os.mkdir(test_dir_dog)
    os.mkdir(test_dir_cat)

    fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname) 
        d = os.path.join(train_dir_cat, fname)
        shutil.copyfile(s, d)   #  把s拷贝到d 

    fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(test_dir_cat, fname)
        shutil.copyfile(s, d)

    fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(train_dir_dog, fname)
        shutil.copyfile(s, d)

    fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(test_dir_dog, fname)
        shutil.copyfile(s, d)

In [5]:
from keras.preprocessing.image import ImageDataGenerator

# 创建图片的迭代器，并且设定它的归一化
train_datagen = ImageDataGenerator(1/255)
test_datagen = ImageDataGenerator(1/255)

In [6]:
# 正式创建图片的生成器：train
train_generator = train_datagen.flow_from_directory( train_dir,                  # 待读取文件的目录
                                                     target_size = (200,200),    # 图片的统一大小 
                                                     batch_size = 20,            # 每次读入20张
                                                     class_mode = 'binary'       # 该文件夹下分两类：因为我已经正好在该文件夹下分了两个文件夹
)

# 正式创建图片的生成器：test
test_generator = test_datagen.flow_from_directory( test_dir,                  # 待读取文件的目录
                                                    target_size = (200,200),    # 图片的统一大小 
                                                    batch_size = 20,            # 每次读入20张
                                                    class_mode = 'binary'       # 该文件夹下分两类：因为我已经正好在该文件夹下分了两个文件夹
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# 使用keras内置VGG16网络：

In [7]:
conv_base = keras.applications.VGG16( weights = 'imagenet', include_top = False, input_shape = (200,200,3))  # 每张图进入都是(200,200,3)

conv_base.trainable = False

In [12]:
# 定义一个：其作用是用卷积基来处理每张图片，获得它们对应的特征提取图，然后把特征提取图存起来
batch_size = 20

# data_generator是生成器，sample_count是样本总数
def extract_features(data_generator, sample_count):
    i = 0
    features = np.zeros( shape=(sample_count, 6, 6, 512) )  # 处理后的结果记录：把每张图变成(6,6,512)的特征提取图 —— 总数(2000,6,6,512)
    labels = np.zeros( shape=(sample_count) )
    # 特征提取 + 存储部分：20张一批一起做 —— (20, 6, 6, 512)
    for inputs_batch, labels_batch in data_generator:
        features_batch = conv_base.predict(inputs_batch)    # 每一张图，经过卷积层处理后，返回其对应的特征提取后的图！ √
        features[ i * batch_size : (i + 1) * batch_size ] = features_batch  # 把特征图存入
        labels[ i * batch_size : (i + 1) * batch_size ] = labels_batch      # 把标签存入
        i += 1
        if i * batch_size >= sample_count:
            break
            
    return features, labels 

In [18]:
# 对训练、测试数据进行特征提取：比较慢！！！s
train_features, train_labels = extract_features(train_generator, 2000)
test_features, test_labels = extract_features(test_generator, 1000)

e:\pycharm\python374\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


### 模型搭建： 

In [14]:
# 不再需要加入那个卷积基了Sequentialquentialquentialquentialquenlayersl接把它特征提取的结果所“对应的尺寸”作为输入即可：
from keras import layers
model = keras.Sequential()
model.add( layers.GlobalAveragePooling2D(input_shape=(6, 6, 512)) )  # 这里输入是特征提取后的尺寸大小。
model.add( layers.Dense(512, activation='relu') )
model.add( layers.Dropout(0.5) )
model.add( layers.Dense(1, activation='sigmoid') )

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 263,169
Trainable params: 263,169
Non-trainable params: 0
_________________________________________________________________


### 模型训练： 

In [16]:
model.compile( optimizer = keras.optimizers.Adam(lr=0.001),
               loss='binary_crossentropy',
               metrics=['acc']
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
# 速度会快非常多！因为只用训练Dense层的参数！！
history = model.fit( train_features, train_labels, epochs=10, batch_size=50, validation_data=(test_features, test_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/10
2000/2000 [==============================] - 2s 864us/step - loss: 0.7175 - acc: 0.8940 - val_loss: 0.1800 - val_acc: 0.9550
Epoch 2/10
2000/2000 [==============================] - 1s 395us/step - loss: 0.2144 - acc: 0.9610 - val_loss: 0.2918 - val_acc: 0.9500
Epoch 3/10
1500/2000 [=====================>........] - ETA: 0s - loss: 0.1516 - acc: 0.9673

KeyboardInterrupt: 